In [ ]:
import numpy as np
from itertools import combinations

class Golay23:
    def __init__(self):
        # Матрица P (12x11) для формирования кода Голея
        self.P = np.array([
            [1,1,0,1,1,1,0,0,0,1,0], [0,1,1,0,1,1,1,0,0,0,1],
            [1,0,1,1,0,1,1,1,0,0,0], [0,1,0,1,1,0,1,1,1,0,0],
            [0,0,1,0,1,1,0,1,1,1,0], [0,0,0,1,0,1,1,0,1,1,1],
            [1,0,0,0,1,0,1,1,0,1,1], [1,1,0,0,0,1,0,1,1,0,1],
            [1,1,1,0,0,0,1,0,1,1,0], [0,1,1,1,0,0,0,1,0,1,1],
            [1,0,1,1,1,0,0,0,1,0,1], [1,1,1,1,1,1,1,1,1,1,1]
        ])
        self.G = np.hstack((np.eye(12, dtype=int), self.P))
        self.H = np.hstack((self.P.T, np.eye(11, dtype=int)))
        
        print("Инициализация: построение таблицы синдромов (может занять пару секунд)...")
        self.syndrome_table = self._build_syndrome_table()

    def _build_syndrome_table(self):
        """Создает словарь: кортеж(синдром) -> вектор_ошибки"""
        table = {}
        n = 23
        # Ошибок может быть 0, 1, 2 или 3
        for r in range(4):
            for indices in combinations(range(n), r):
                e = np.zeros(n, dtype=int)
                e[list(indices)] = 1
                s = np.dot(e, self.H.T) % 2
                table[tuple(s)] = e
        return table

    def encode(self, data):
        return np.dot(data, self.G) % 2

    def decode(self, received):
        # 1. Вычисляем синдром: S = r * H^T
        s = np.dot(received, self.H.T) % 2
        s_tuple = tuple(s)
        
        if np.all(s == 0):
            return received[:12]
        
        # 2. Ищем вектор ошибки в таблице
        if s_tuple in self.syndrome_table:
            e = self.syndrome_table[s_tuple]
            # 3. Исправляем: исправленное = (полученное + ошибка) mod 2
            corrected = (received + e) % 2
            return corrected[:12]
        else:
            print("Слишком много ошибок (больше 3-х)!")
            return received[:12]


In [ ]:
golay = Golay23()

# Исходное сообщение 12 бит
msg = np.array([1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0])

# Кодирование
codeword = golay.encode(msg)

# Вносим ровно 3 ошибки в случайные места
received = codeword.copy()
error_indices = [2, 10, 20] 
for idx in error_indices:
    received[idx] ^= 1
    
print(f"Исходное:     {msg}")
print(f"С ошибками:   {received.astype(int)}")

# Декодирование
decoded = golay.decode(received)
print(f"Исправленное: {decoded.astype(int)}")

if np.array_equal(msg, decoded):
    print("Результат: ОШИБКИ ИСПРАВЛЕНЫ УСПЕШНО!")
else:
    print("Результат: Ошибка восстановления.")